In [ ]:
import pandas as pd
import numpy as np 
import altair as alt

In [ ]:
url = 'https://github.com/byuidatascience/data4missing/raw/master/data-raw/flights_missing/flights_missing.json'

flights = pd.read_json(url)

In [ ]:
namonth = flights.query('month == "n/a"')
namonth.count()

In [ ]:
worst = (flights
    .groupby('airport_code')
    .agg(total_flights = ('num_of_flights_total', sum),
        total_delays = ('num_of_delays_total', sum),
        total_delay_min = ('minutes_delayed_total', sum))
    .assign(percent_delays = lambda x: x.total_delays / x.total_flights)
    .assign(avg_delay_hrs = lambda x: (x.total_delay_min / x.total_delays) / 60)
    .reset_index()
    )

print(worst.to_markdown(index = False))      ##Copy and paste result into report##

In [ ]:
delay_hrs = (alt.Chart(worst)
    .mark_bar().encode(
     x = alt.X('airport_code', axis = alt.Axis(title = "Airport")),
     y = alt.Y('avg_delay_hrs', axis=alt.Axis(title = "Average Delay")))
    .properties(
    height = 400,
    width = 400,
    title = {'text': "Longest Delay Times", 'subtitle': "By Airport"}))

delay_hrs

In [ ]:
delay_hrs.save('delay_hrs.png')

Question 2
What is the best month to fly if you want to avoid delays of any length?

In [ ]:
#remove n/a columns
flight_month = flights.query("month != 'n/a'")

worst2 = (flight_month
    .groupby('month')
    .agg(total_flights = ('num_of_flights_total', sum),
        total_delays = ('num_of_delays_total', sum),
        total_delay_min = ('minutes_delayed_total', sum))
    .assign(percent_delays = lambda x: x.total_delays / x.total_flights)
    .assign(avg_delay_hrs = lambda x: (x.total_delay_min / x.total_delays) / 60)
    .reset_index()
    )

month_list = ['January', 'Febuary', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

worstmonth = (alt.Chart(worst2).mark_bar()
    .encode(x = alt.X('month', sort = month_list, axis = alt.Axis(title = "")),
            y = alt.Y('percent_delays', axis = alt.Axis(format = '%', title = "")))
    .properties(
    height = 400,
    width = 400,
    title = {'text': "Percent of Flights Delayed", 'subtitle': "By Month"}))


In [ ]:
worstmonth.save('worstmonth.png')

In [ ]:
#total_weather_delays = weather + 0.3 * LateAircraft + {if April-August, 0.4 * NAS else 0.65 * NAS}#

#replacing -999 with NaN
q3data = flights

q3data.num_of_delays_late_aircraft.replace(-999, np.nan, inplace = True)

#Calculate mean
q3mean = q3data.num_of_delays_late_aircraft.mean()

#Replace NaN with mean
q3data.num_of_delays_late_aircraft.replace(np.nan, q3mean, inplace = True) #or q3data.num_of_delays_late_aircraft.fillna(value = q3mean, inplace = True)

In [ ]:
months = ["April", "May", "June", "July", "August"]

weather = q3data.assign(
    severe = q3data.num_of_delays_weather,
    mild_late = .3*q3data.num_of_delays_late_aircraft,
    mild_nas = np.where(q3data.month.isin(months), .4 *q3data.num_of_delays_nas, .65 * q3data.num_of_delays_nas),
    total = lambda x: x.severe + x.mild_late + x.mild_nas)

In [ ]:
weatherfiltered = weather.filter(['airport_code', 'severe', 'mild_late', 'mild_nas', 'total']).head()

print(weatherfiltered.to_markdown(index = False))

In [ ]:
#weather_proportion = weather.assign(
#       weather_late_proportion = weather.total / weather.num_of_flights_total)

percent_delayed_w = (weather.groupby("airport_code")
  .agg(total_weather =  ('total', sum),
      total_flights = ('num_of_flights_total', sum))
  .assign(percent_delay_w = lambda x: x.total_weather / x.total_flights)
  .reset_index()
)

In [ ]:
weatherpercent = (alt.Chart(percent_delayed_w).mark_bar()
    .encode(y = alt.Y('airport_code', axis = alt.Axis(title="Airport Code")),
            x= alt.X('percent_delay_w', axis = alt.Axis(format = '%', title = "Percent Delayed")))
       .properties(
        height = 400,
        width = 400,
        title = {'text': "Airport with highest percentage of late flights", 'subtitle': "Due to weather"})
        )

In [ ]:
weatherpercent.save('weatherpercent.png')

In [ ]:
#Fix missing data, then export as json
replace_list = [-999, 'n/a',"","1500+"]

q5data = flights.replace(replace_list, np.nan)

import json

json_data = q5data.to_json(orient = "records")

json_object = json.loads(json_data)
json_formatted_str = json.dumps(json_object, indent = 4)

print(json_formatted_str)